In [1]:
import json
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import random

In [ ]:
#optional cell if using drive:
#if using drive the file creeation will not work!
#from google.colab import drive
#drive.mount('/content/gdrive')



# Section 0: Getting data from million songs playlists

### How to use this notebook

1. Download it
2. Download and extract the million songs dataset (link in the README.md on the repo) in the same directory as this notebook
3. Set the DIRECTORY constant in the cell below as the directory containing the dataset
4. You should then be able to run all of it

Section 1 is making a graph to figure out how the number of followers over all the playlists is distributed
Section 2 is removing all playlists from the dataset which have N or more fewer, and creating a new file containing playlists with only more than N followers
Section 3 is extracting N songs randomly from a given dataset, either randomly or with a weighted random distribution. You can set the constants to change the sampling method, number of songs, and the dataset you are computing over. If you use the default dataset to pick the songs, it is quite slow (takes like 10 minutes overall), so you may want to use a smaller dataset to choose these songs. I set the dataset to the dataset containing playlists with over 5 followers and it is significantly quicker to run. 

### IMPORTANT: Set the directory containing the data below

In [2]:
DIRECTORY = "spotify_million_playlist_dataset/data/"

### 1. Figuring out distribution of number of followers over playlists

This is just to figure out what the distribution of the followers looks like

In [ ]:
# 1. Followers numbers
# 10,000 songs choose (Quanchi's link)


num_followers = []
directory = os.listdir(DIRECTORY)

for file in directory:

    f = open(DIRECTORY+file)
    data = json.load(f)


    for playlist in range(len(data['playlists'])):
        x = data['playlists'][playlist]['num_followers']
        num_followers += [x]

num_followers = pd.Series(num_followers)
num_followers.value_counts()


In [ ]:
num_likes = {}

for val in num_followers:
    if val in num_likes:
        num_likes[val] += 1
    else:
        num_likes[val] = 1

keys = []
for key in num_likes.keys():
    keys += [key]

keys.sort()

X,y = [],[]

for key in keys:
    X +=[key]
    y +=[num_likes[key]]
    
    
#make the y-values a fraction of the size of the dataset
y_c = y     
for i in range(len(y_c)):
        y_c[i] = (y_c[i]+y_c[i-1])

for i in range(len(y_c)):
    y_c[i]/= 1000000

y_c


    
    

In [ ]:
#Plot

plt.plot(X[:40], y[:40],linestyle="-", marker="o")
plt.ylabel("Fraction of playlists with followers <= than this number")
plt.xlabel("Number of Followers")
plt.xticks(np.arange(0,41,2))
plt.show()
f = plt.figure()
f.set_figwidth(1)
f.set_figheight(1)

In [ ]:
print(1000000 - y_c[5]*1000000)


### 2. Cleaning the data (Method 1)

Removing all playlists with N or less followers 

In [3]:
N = 10
directory = os.listdir(DIRECTORY)

moreThanN = []

for file in directory:

    f = open(DIRECTORY+file)
    data = json.load(f)


    for playlist in range(len(data['playlists'])):
        if data['playlists'][playlist]['num_followers'] >N:  
            moreThanN += [data['playlists'][playlist]]

In [4]:
#sanity check

len(moreThanN)

6512

In [5]:
#Create results folder

path = os.getcwd()+"/generated_data"
# Check whether the specified path exists or not
isExist = os.path.exists(path)
if not isExist:

   # Create a new directory because it does not exist
   os.makedirs(path)
   print("The new directory is created!")

The new directory is created!


In [6]:
#SAVING DATA
#convert to dictionary 
moreThanN_dict = {"playlists": moreThanN}

json_object = json.dumps(moreThanN_dict, indent=4)
with open(path+"/moreThan"+str(N)+"followers.json", "w") as outfile:
    outfile.write(json_object)



In [7]:
edges = {} #will store edges
songs_dict = {} #assign a number to each song


f = open(path+"/moreThan"+str(N)+"followers.json")
data = json.load(f)
song_number = 0        
count = 0
for playlist in data['playlists']:
#     r = random.randint(1,100) #This line selects a random percent of all the playlists, in case the .edges file is too big still
#     if r<80:
#         continue
    edge_weight = playlist['num_followers']
    if (count % 1000 == 0):
        print(count)
    count = count +1
    for track1 in playlist['tracks']:      
        
        
        for track2 in playlist['tracks']:
            
            if track1['track_name'] not in songs_dict:
                songs_dict[track1['track_name']] = str(count)
                count += 1
            if track2['track_name'] not in songs_dict:
                songs_dict[track2['track_name']] = str(count)
                count += 1
            
            t1 = songs_dict[track1['track_name']]
            t2 = songs_dict[track2['track_name']]
            
            if t1 != t2:  #if they are not the same track
                if (t1 + ' ' + t2) in edges:
                    edges[(t1 + ' ' + t2)] += edge_weight
                elif (t2 + ' ' + t1) in edges:
                    edges[(t2 + ' ' + t1)] += edge_weight
                else:
                    edges[(t1 + ' ' + t2)] = edge_weight

                    
print(len(edges))


                     
                 
                 
            
         


0
13700
18000
23500
27400
36900
38600
39200
39900
40200
40300
47300
48600
56100
67000
67400
68200
70500
75700
78100
81100
83200
84300
87600
92100
94800
96300
96700
97500
97900
98100
98500
99100
100000
100500
101200
102500
104300
106300
109700
112300
114200
117400
120700
121100
121900
122100


KeyboardInterrupt: 

In [ ]:
#formatting stuff for .edges filef

data = ""

for key in edges:
    data+= (key + ' ' + str(edges[key])+'\n')

In [ ]:
#writing the data

with open(path+"/moreThan"+str(N)+"followers.edges", "w") as outfile:
    outfile.write(data)

### 3. Cleaning the data (Method 2)

Selecting N songs randomly and performing an analysis on them

Change the constants in the cell below to change the kind of data you create

In [53]:
################# Constants #################
'''Rename this constant to the file containing the data, otherwise it will go through all of the data.
I recommend using a file generated by Method 1 as the specific file (for instance, moreThan10followers.json) since
the computation times will be much much faster'''
SPECIFIC_FILE = 'generated_data/moreThan10followers.json' 

WEIGHTED_RANDOM = False   #If we want weighted random sampling or not
N = 10000  #number of songs to consider

############################################

In [54]:

#first we need to load all the songs into a dictionary so that we can randomly select N of them


all_tracks = {} #stores all the songs
count = 0 #progress bar


if SPECIFIC_FILE == '':
    directory = os.listdir(DIRECTORY)
    for file in directory:
        count +=1
        if (count %100 == 0):
            print(count)
        f = open(DIRECTORY+file)
        data = json.load(f)

        for playlist in data['playlists']:
            for track in playlist['tracks']:
                if track['track_name'] not in all_tracks:  #if it not in the dict then we add it to it and increase its weight
                    all_tracks[track['track_name']] = 1 #weight is used for weighted random sampling (if needed)
                else:
                    all_tracks[track['track_name']] +=1
                
############### TO LOOP OVER SOME SMALLER FILE ##################\
else:
    f = open(SPECIFIC_FILE)
    data = json.load(f)

    for playlist in data['playlists']:
        for track in playlist['tracks']:
            if track['track_name'] not in all_tracks:  #if it not in the dict then we add it to it and increase its weight
                all_tracks[track['track_name']] = 1 #weight is used for weighted random sampling (if needed)
            else:
                all_tracks[track['track_name']] +=1



In [55]:
print(list(all_tracks.keys())[0:20])

['T-Shirt', 'It All Started with a Beer', "Night's On Fire", 'From the Ground Up', 'Crash And Burn', 'Break Up with Him', "I Don't Dance", 'Dirt', 'Die A Happy Man', 'It Goes Like This', "Don't It", '19 You + Me', "Nothin' Like You", 'I Know Somebody', "It Don't Hurt Like It Used To", 'Vacation', 'Different For Girls', 'Sleep Without You', 'Star Of The Show', 'Sunshine & Whiskey']


In [56]:
#get N songs based on weighted random sampling (no particular reason for this apart from probably giving a more connected NW)

if WEIGHTED_RANDOM == False:
    songs = random.choices(list(all_tracks.keys()), weights=None, k=N) 
else:
    songs = random.choices(list(all_tracks.keys()), weights=all_tracks.values(), k=N)

#Here we assign int indices to each song (to make the .edges file later we need nodes as int values)
temp = {}
index = 1
for song in songs:
    if song not in temp:
        temp[song] = index
        index +=1
songs = temp

print(songs)

{"Let's Start From Here": 1, 'Antes De Que Nos Olviden': 2, "L'amour": 3, 'Invincible - Ducked Ape Remix': 4, 'Madly In Love With You': 5, 'Cannibal Queen': 6, 'To Build A Home - Grey Reverend Version': 7, 'In Common - Kaskade Radio Mix': 8, 'S.O.S.': 9, 'Lovely Ladies': 10, 'Electronic Drops': 11, 'If I Die Young - Dubstep Remix': 12, "My Feet Can't Fail Me Now": 13, 'Oblighetto': 14, 'Shadow Dances': 15, 'Castle On A Cloud': 16, 'Next to You': 17, 'No Mistaking Love (A Tribute to Mel & Violet Kraemer)': 18, 'Sun Medallion': 19, 'El Lay': 20, 'Star Faces': 21, 'The Banishment Song': 22, 'Stop for Nothing': 23, 'Here is to you - the ballad of Sacco and Vanzentti': 24, 'Rudolph, The Red-Nosed Reindeer - Malibu Remix': 25, 'Maleah': 26, 'Sideways (feat. Marco Foster)': 27, 'You Have My Surrender (Live)': 28, 'God Of Brilliant Lights': 29, 'Did You Really Know': 30, 'Two Thousand and Seventeen': 31, 'Little Dark Age': 32, 'Follow You (Virtual Riot Remix) [feat. Danyka Nadeau]': 33, 'Count

In [57]:
#Now I want to create a list of shared playlists

shared_playlists = [] #this will contain items of the form (weight: [weight], tracks: [tracks]) for each playlist

count = 0 #Cosmetic, to show progress


if SPECIFIC_FILE == '':
    for file in directory:
        f = open(DIRECTORY+file) 
        data = json.load(f)

        for playlist in data['playlists']:  #parse through every playlist

            count+=1
            if (count%10000==0):
                print(count)

            num_followers = playlist['num_followers']

            temp = []

            for track in playlist['tracks']:
                if track['track_name'] in songs:
                    index = songs[track['track_name']]
                    if index not in temp:
                        temp.append(index)

            if (len(temp) >1): #only if a playlist has two or more of the 10k songs then we add it to our shared_playlists lits
                item = {'weight':num_followers, 'tracks':temp}
                shared_playlists.append(item)

else:
    
    f = open(SPECIFIC_FILE) 
    data = json.load(f)

    for playlist in data['playlists']:  #parse through every playlist

        count+=1
        if (count%10000==0):
            print(count)

        num_followers = playlist['num_followers']

        temp = []

        for track in playlist['tracks']:
            if track['track_name'] in songs:
                index = songs[track['track_name']]
                if index not in temp:
                    temp.append(index)

        if (len(temp) >1): #only if a playlist has two or more of the 10k songs then we add it to our shared_playlists lits
            item = {'weight':num_followers, 'tracks':temp}
            shared_playlists.append(item)

            
                

In [58]:
#create edges
edges = {}
count = 0
for item in shared_playlists:
    if (count%10000 == 0):
        print(count)
    count +=1
    weight = item['weight']
    tracks = item['tracks']
    for track1 in tracks:
        for track2 in tracks:
            if (track1 == track2):
                continue
            if (str(track1) + ' ' + str(track2)) in edges:
                edges[str(track1) + ' ' + str(track2)] += weight
            elif (str(track2) + ' ' + str(track1)) in edges:
                edges[str(track2) + ' ' + str(track1)] += weight
            else:
                edges[str(track1) + ' ' + str(track2)] = weight
            

0


In [59]:
data = ""

for key in edges:
    data+= (key + ' ' + str(edges[key])+'\n')

print(data) #sanity check

6300 765 1242
6300 7889 1168
6300 171 198
6300 406 886
6300 2424 958
6300 5481 58
765 7889 1120
765 171 276
765 406 908
765 2424 1152
765 5481 58
7889 171 92
7889 406 886
7889 2424 972
7889 5481 58
171 406 58
171 2424 114
171 5481 230
406 2424 976
406 5481 96
2424 5481 126
2694 7539 450
2694 8362 396
2694 4037 16614
2694 6030 218
2694 9078 16594
2694 9570 220
2694 4382 23426
2694 475 220
2694 1128 184
7539 8362 528
7539 4037 608
7539 6030 220
7539 9078 362
7539 9570 184
7539 4382 656
7539 475 184
7539 1128 184
8362 4037 424
8362 6030 184
8362 9078 210
8362 9570 208
8362 4382 730
8362 475 184
8362 1128 184
4037 6030 206
4037 9078 16276
4037 9570 244
4037 4382 16518
4037 475 184
4037 1128 184
6030 9078 220
6030 9570 184
6030 4382 236
6030 475 184
6030 1128 184
9078 9570 184
9078 4382 16260
9078 475 184
9078 1128 184
9570 4382 248
9570 475 184
9570 1128 184
4382 475 184
4382 1128 184
475 1128 184
8413 8977 28
8413 2270 28
8413 4807 28
8413 9459 28
8413 1411 28
8413 2293 28
8413 3914 28
84

In [60]:
#Create results folder

path = os.getcwd()+"/generated_data"
# Check whether the specified path exists or not
isExist = os.path.exists(path)
if not isExist:

   # Create a new directory because it does not exist
   os.makedirs(path)
   print("The new directory is created!")

In [61]:
#Writing the file

name = ""
if (WEIGHTED_RANDOM == True):
    name = str(N) + '_weighted'
else:
    name = str(N)

with open(path+'/'+name+"_random_songs.edges", "w") as outfile:
    outfile.write(data)
    
    
#The data should be saved in the same folder as this notebook

## Thoughts so far

- Selecting playlists above a certain number of likes produces a very dense network even with only the top 600~ networks
- Selecting 10k songs randomly produces a pretty sparse network with some intereseting communities
- Selecting 10k songs randomly with a weighted distribution produces an extremely dense network
- The clustering coefficient for 100 random weighted songs is small (0.04), but when we pick 500 random weighted songs it grows larger (0.34). For 10000 random songs it was like 0.99

# Section 1: Defining the edges of the network

In this section we explore several different means to quantify how similar one song is to another. We will use these similarity scores to assign weights to the edges of our network. 

### Idea 1: Shared Followers
This method assigns a weight to each edge based on the sum of likes of each playlist that both songs in the edge appear in.

$w_{i,j} = \Sigma_{p \in P_{i,j}}{\normalsize \text{# likes of }p}$, where $P_{i,j}$ is the set of playlists contatining songs $i$ and $j$

### Idea 2: Shared Playlists
The weight of each edge is the number of shared playlists of the song

$w_{i,j} = \normalsize |P_{i,j}|$

### Idea 3: Hybrid of 1 and 2
Since the follower numbers of the playlists are heavily skewed, ranging from 1 to over 70k, we experiment with weighting the shared playlist number higher than the shared follower number.

$w_{i,j} = \normalsize |P_{i,j}| + log_2({\Sigma_{p \in P_{i,j}}{\text{# likes of }p}})$

For our purposes, we use $log_2$ to weight the follower numbers as we think it suits our data well.

### Idea 4: Shared Playlist Simliarity

Let $p_{i,j}$ denote the fraction of playlists that song $i$ belongs to which also contain song $j$. 
Then, $ w_{i,j} = \large \frac{p_{i,j}\cdot p_{j,i}}{2}$

I.e, this is the average of the fraction of all playlists containing one song that also contain the other


## TODO: Define functions that implement each of these ideas

Each function should take in an input file (not the whole dataset as that is way too big!) and return two things: 1. The edges of the graph created by each method, and 2. A dictionary of node relations to each other.


# Section 2: Implementing algorithm to generate playlists

## 2.1: Star Graph Algorithm

Here I'll try to create the playlists using the star graph recommendation method. For this I need two things - first is a recommendation algorithm based on the edges, and second is an easy way to iterate over song connections. 

### How this works:

First we consider the one node case. We feed a single node, call it node A, into the algorithm which gives us the subset of the network with one edge at A, which is a star graph. We store this star graph in a dictionary object. The next song to be recommended is one of the edges in this graph, randomly chosen with a weighted probability distribution. Call this song B. Now, we want the playlist that we are generating to be the center of the star graph. Now, we make the center of the star graph represent both nodes A and B. We add all the edges with end to B (except those with end A) from the original dataset to the star graph. If an edge already exists between the center and another node, and B is also connected with that node, the new weight of that edge becomes the sum of the weights of those edges. This is to implement familiarity. Continue this until the desired playlist size is reached. 


### TODO:
- If we can create functions for creating the edge weights of the network that would be totally awesome
- Perhaps cosine simliarity would be a good thing to explore for our recommendations?

Running the two cells below will initialize the network. There are some constants at the start that you need to input

In [ ]:
######################################### Constants

SPECIFIC_FILE = 'generated_data/moreThan10followers.json' #Rename this to something else if you want to only consider some other dataset, 
                    #otherwise we go over all data. specify the file name/path as the name
WEIGHTED_RANDOM = False   #If we want weighted random sampling or not
N = 1000  #number of songs to consider

In [ ]:
######################################### LOADING SONGS FROM FILE
#first we need to load all the songs into a dictionary so that we can randomly select N of them


all_tracks = {} #stores all the songs
count = 0 #progress bar


f = open(SPECIFIC_FILE)
data = json.load(f)

for playlist in data['playlists']:
    for track in playlist['tracks']:
        if track['track_name'] not in all_tracks:  #if it not in the dict then we add it to it and increase its weight
            all_tracks[track['track_name']] = 1 #weight is used for weighted random sampling (if needed)
        else:
            all_tracks[track['track_name']] +=1


######################################### CHOOSING RANDOM SONGS

if WEIGHTED_RANDOM == False:
    songs = random.choices(list(all_tracks.keys()), weights=None, k=N)
else:
    songs = random.choices(list(all_tracks.keys()), weights=all_tracks.values(), k=N)

#print(songs)

######################################### CREATING SHARED PLAYLIST EDGES

shared_playlists = [] #this will contain items of the form (weight: [weight], tracks: [tracks]) for each playlist


f = open(SPECIFIC_FILE) 
data = json.load(f)

for playlist in data['playlists']:  #parse through every playlist

    num_followers = playlist['num_followers']
    temp = []
    for track in playlist['tracks']: #if one of the random songs is in the playlist then remember it
        if track['track_name'] in songs and track['track_name'] not in temp:
            temp.append(track['track_name'])

    if (len(temp) >1): #only if a playlist has two or more of the 10k songs then we add it to our shared_playlists lits
        item = {'weight':num_followers, 'tracks':temp}
        shared_playlists.append(item)


# #Here we create the big graph
edges = {}
relations = {} #this variable stores all of the node pairs in the graph. It is useful for the playlist-generation part of the notebook
for song in songs:
    relations[song] = set({})

for item in shared_playlists:
    
    weight = item['weight']
    tracks = item['tracks']
    for track1 in tracks:
        for track2 in tracks:
            if (track1 == track2):
                continue
            #now we add relations
            if track2 not in relations[track1]:
                relations[track1].add(track2)
            if track1 not in relations[track2]:
                relations[track2].add(track1)
                
            #now create edge and weight
            if (track1,track2) not in edges and (track2,track1) not in edges:
                edges[(track1,track2)] = weight
                edges[(track2,track1)] = weight
            else:
                edges[(track1,track2)] += weight
                edges[(track2,track1)] += weight


print("Some songs that were selected:")
print(songs[0:5])
print('\n')
print("Some of the edges that were created:")
print(list(edges.items())[0:5])
print('\n')
print("Some of the pairs of nodes in the graph:\n")
for n in range(5):
    i = random.randint(0,len(songs))
    print("Track:"+str(songs[i]))
    print("Tracks it is related to:")
    print(relations[songs[i]])
    print('\n')





The cell below defines the function that we use

In [ ]:
def recommend_song(stargraph, songs, alpha = 1):
    '''
    Function that will implement the recommend algorithm for a given input star graph. We are assigning weights to every 
    song in the number of songs we are operating over. The weight of each song will be the weight of the edge from it to the
    center of the stargraph (0 otherwise) + alpha. We will then make a weighted random prediction to get the next song.
    
    Parameters:
        stargraph: the input star graph. The first element is the nodes inside the graph and the second one is a dictionary 
        of node:weight pairs
        songslist: the list of songs over which we are operating
        alpha: value to give to every song to allow possible recommendation of songs without any edges (teleportation). 1 by default
  
  Returns:
       The predicted song
   '''
    weights = []
    for i in range(len(songs)):
        song = songs[i]
        weight = alpha
        if song in stargraph[1]: #if there is already an edge to it
            weight+= stargraph[1][song]
        if song in stargraph[0]: #if it is already in our playlist
            weight = 0
        weights.append(weight)
    nextsong = random.choices(songs, weights, k=1)[0] #return our weighted random choice
    #print (nextsong)
    return nextsong

def update_graph(stargraph, songToAdd, relations, edges):
    '''
    Function that will update the star graph to include edges that belong to the song to add. Returns updated star graph
    
    Parameters:
        stargraph: the input star graph. The first element is the nodes inside the graph and the second one is a dictionary 
        of node:weight pairs
        songToAdd: the song to add
        relations: the dictionary of all pairs in the graph
        edges: the original network with all the node pair relations
   
   Returns:
        The updated star graph
    '''
    relationsToAdd = relations[songToAdd] #give a set of all the pairs of nodes with the song to add
    stargraphNodes = stargraph[0]
    stargraphEdges = stargraph[1]
    
    for song in relationsToAdd: #go through each pair belonging to this song
        
        if song not in stargraphNodes:
            if song not in stargraphEdges:
                stargraphEdges[song] = edges[(song,songToAdd)] #add value of edge from the original network
            else:
                stargraphEdges[song] += edges[(song,songToAdd)]
                
    stargraph[0].add(songToAdd)
    return stargraph
    
    
def create_playlist(inputSongs, length,  songs, relations, edges,alpha = 1):
    '''
    Function that generates a recommendation playlist of length length based on an input of songs, and an input network
    
    Parameters:
    inputSongs: The input songs (list)
    length: Number of songs to add
    songs: List of all songs
    relations: Dictionary of all pair relations in the graph
    edges: The original network
    alpha: The teleportation probability in the recommendation algorithm
    '''
    stargraph = (set(), dict())
    
    for song in inputSongs: #build the star graph
        stargraph = update_graph(stargraph, song, relations, edges) 
    
    for i in range(length):
        newSong = recommend_song(stargraph, songs, alpha)
        stargraph = update_graph(stargraph, newSong, relations, edges)
    
    return list(stargraph[0])
    
    
            
        
        
    
    

## Using the algorithm

Finally, the cell below tests the algorithm

In [ ]:
#Testing the algorithm

playlist = []
for i in range(3):
    playlist.append(songs[random.randint(0,len(songs))])
length = len(playlist)+10
print("Input songs:")
print(playlist)
playlist = create_playlist(playlist, length, songs, relations, edges)
print("Generated playlist:")
print(playlist)


### Thoughts on algorithm

I have no idea if these are great playlists or not lmao

## 2.2 Cosine Simliarity

### TODO
- maybe this would be good for idea #4 of Section 1 since the weights are all normalized and in the [0,1] range